In [1]:
# FIX BUG WITH SQLITE
# https://gist.github.com/defulmere/8b9695e415a44271061cc8e272f3c300

__import__('pysqlite3')
import sys
import os
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join('..', 'db.sqlite3'),
    }
}

In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.schema.document import Document

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [4]:
loader = CSVLoader(file_path=os.path.join(sys.path[0], "../src/artifacts/@karpovcourseschat.csv"), source_column="text", metadata_columns=["date", "message_id"], csv_args={
    "delimiter": ";",
})
data = loader.load()

In [5]:
load_dotenv(dotenv_path='../.env')
PROXY_API_KEY = os.getenv("PROXY_API_KEY")

In [6]:
emb_fn = OpenAIEmbeddings(
    api_key=os.getenv("PROXY_API_KEY"),
    model="text-embedding-ada-002",
    base_url="https://api.proxyapi.ru/openai/v1",
)

/home/kartushov/pet_projects/PostFinder/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [7]:
db = Chroma.from_documents(data, emb_fn)

In [8]:
from langchain.chat_models import ChatOpenAI

In [9]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=PROXY_API_KEY, openai_api_base="https://api.proxyapi.ru/openai/v1")

/home/kartushov/pet_projects/PostFinder/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [32]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question", "context"],
    template="""Answer the question based on the context below. If the context doesn't contain the answer, say "I don't know."\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:""",
)

In [27]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = db.as_retriever()

In [28]:
question = "Что такое симулятор мл?"

In [29]:
docs = retriever.get_relevant_documents(question)

In [42]:
context_text = "\n\n---\n\n".join([doc.page_content for doc in docs[::-1]])

In [45]:
prompt =QUERY_PROMPT.format(context=context_text, question=question)

In [49]:
llm.predict(prompt)

/home/kartushov/pet_projects/PostFinder/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Симулятор МЛ - это программное обеспечение или игра, которая позволяет практиковать и оттачивать навыки в области машинного обучения.'